## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [2]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_0 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o0/o0_bincore8.csv", index_col=0)
print(bin8_0.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_0.reset_index(inplace=True, drop=True)

print('shape -', bin8_0.shape)
print('reset_index 완료')
print('input data shape')
bin8_0.head()

(22459287, 2) (7651934, 2)
bin - core concat
shape - (30111221, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [3]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_0['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_0['label'].value_counts())

256
0    30060985
1       50236
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [4]:
idx_bin = bin8_0[bin8_0['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_0), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_0', len(ls_idx_bin))

# loc 로 수정필요
bin8_0_Ngram = bin8_0.loc[ls_idx_bin,:].copy()

1607552
나머지 0
최종 길이 1607552
bin8_0 1607552


## (4) false data 만들기

In [5]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_0_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_0.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_0)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_0[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

79730.0
0 3921410
1000 16506259
2000 10690157
3000 29609676
4000 6548859
5000 19642312
6000 3494053
7000 24726310
8000 26637051
9000 29206116
10000 24207739
11000 23873191
12000 7546867
13000 26707276
14000 6435133
15000 1186544
16000 1144639
17000 11175863
18000 12905527
19000 6725049
20000 12062965
21000 11907480
22000 3548298
23000 1797704
24000 6107715
25000 12869785
26000 17160316
27000 18271149
28000 97301
29000 11539523
30000 1606489
31000 17132411
32000 23176911
33000 23715084
34000 8221123
35000 11672321
36000 20427516
37000 7826810
38000 22343980
39000 6378052
40000 7078186
41000 9402546
42000 2812503
43000 2443202
44000 19290372
45000 3800424
46000 15615910
47000 18793925
48000 26969786
49000 20752673
50000 10658342
51000 9827857
52000 1795828
53000 19475504
54000 7266147
55000 12909671
56000 6733053
57000 1654260
58000 9633854
59000 18683943
60000 19122650
61000 18457535
62000 15442641
63000 29557399
64000 12772279
65000 26632343
66000 21654553
67000 4783973
68000 10913957


In [6]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_0['label'].value_counts()[1])

79730
50236


## (5) False Data + True Data 합치기

In [7]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_0_Ngram])
final.shape

(5102720, 2)

## (6) one hot encoding

In [8]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_0_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_0_onehot_Ngram = pd.concat([final['label'], bc8_0_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_0_onehot_Ngram.shape)

원핫인코딩완료
(5102720, 257)


In [9]:
# 훈련 데이터, 훈련 라벨
x_bc8_0 = bc8_0_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_0 = bc8_0_onehot_Ngram['label'].to_numpy()
print(x_bc8_0.shape, x_bc8_0.shape)

x_bc8_0 = x_bc8_0.reshape(-1, right_idx, x_bc8_0.shape[1])
y_bc8_0 = y_bc8_0.reshape(-1, right_idx, 1)

print(x_bc8_0.shape, y_bc8_0.shape)

(5102720, 256) (5102720, 256)
(159460, 32, 256) (159460, 32, 1)


In [10]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_0.shape[0])

x_bc8_0 = x_bc8_0[p]
y_bc8_0 = y_bc8_0[p]

print(x_bc8_0.shape, y_bc8_0.shape)

(159460, 32, 256) (159460, 32, 1)


## (7) 모델

In [11]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [12]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_0, y_bc8_0):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_0[train],
               y_bc8_0[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_0[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_0[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 121s 846us/sample - loss: 0.0040 - accuracy: 0.9991
Epoch 2/10
143514/143514 [==============================] - 100s 695us/sample - loss: 4.8301e-04 - accuracy: 0.9999
Epoch 3/10
143514/143514 [==============================] - 96s 666us/sample - loss: 3.5885e-04 - accuracy

accuracy_score 0.9998980935657845
recall_score 0.9946790250600756
precision_score 0.9963892709766162
f1_score 0.9955334135028344
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 109s 760us/sample - loss: 1.8505e-04 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 96s 668us/sample - loss: 1.

accuracy_score 0.9999353286090555
recall_score 0.9967736457802683
precision_score 0.9976206662134602
f1_score 0.9971969761318271
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 52s 361us/sample - loss: 1.1909e-04 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 45s 315us/sample - loss: 1.1

accuracy_score 0.9999725636523266
recall_score 0.9993162393162394
precision_score 0.9982923497267759
f1_score 0.9988040321202801
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 48s 335us/sample - loss: 5.7003e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 46s 319us/sample - loss: 4.7

accuracy_score 0.9999823623479243
recall_score 1.0
precision_score 0.998470168281489
f1_score 0.9992344985965809
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 39s 269us/sample - loss: 2.6259e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 31s 215us/sample - loss: 2.4376e-05 - accura

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 53s 372us/sample - loss: 3.3208e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 43s 298us/sample - loss: 3.3207e-06 - accuracy: 1.0000
Epoch 3/10
143514/143514 [=======

accuracy_score 0.9999980402608805
recall_score 1.0
precision_score 0.99983119513842
f1_score 0.9999155904448384
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 38s 264us/sample - loss: 6.6411e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 31s 213us/sample - loss: 6.6411e-06 - accurac

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 39s 273us/sample - loss: 6.6411e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 33s 230us/sample - loss: 6.6411e-06 - accuracy: 1.0000
Epoch 3/10
143514/143514 [=======

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 46s 323us/sample - loss: 6.6411e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 33s 227us/sample - loss: 6.6411e-06 - accuracy: 1.0000
Epoch 3/10
143514/143514 [=======

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9998980935657845, 0.9999353286090555, 0.9999725636523266, 0.9999823623479243, 0.9999941207826414, 1.0, 0.9999980402608805, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9946790250600756, 0.9967736457802683, 0.9993162393162394, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9963892709766162, 0.9976206662134602, 0.9982923497267759, 0.998470168281489, 0.9995007488766849, 1.0, 0.99983119513842, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9955334135028344, 0.9971969761318271, 0.9988040321202801, 0.9992344985965809, 0.9997503121098626, 1.0, 0.9999155904448384, 1.0, 1.0, 1.0]


## (9) 평가지표

In [13]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999780509218613
10-Fold Cross_validation. Recall : 0.9990768910156582
10-Fold Cross_validation. Precision : 0.9990104399213446
10-Fold Cross_validation. F1-Score : 0.9990434822906223


In [14]:
model1.save('gcc8_bin_core_s32_h48_o0.h5')
print('save 완료')

save 완료
